# https://github.com/kernoelpanic/sccc

# Smart Contract CrashCourse

This is an introduction to Smart Contracts based on Ethereum, Ganache and Web3py.
This tutorial is is part of the [Cryptocurrencies lecture](https://tiss.tuwien.ac.at/course/courseDetails.xhtml?dswid=1923&dsrid=980&courseNr=192065&semester=2018W) at TU Wien.

## Ethereum

Ethereum is a smart contract platform, there are also others (e.g., [Cardano](https://www.cardano.org/en/home/),[hyperleder](https://www.hyperledger.org/),[rootstock](https://www.rootstock.com/),[quorum](https://github.com/jpmorganchase/quorum), ...) but Ethereum is the most common (and I dont know as much about the others :) Therefore the tutorial as well as the project/exercise will focus on Ethereum style smart contracts. 

**Note:** A lot will (is supposed to) change in future ethereum versions i.e., [ETH2.0](https://docs.ethhub.io/ethereum-roadmap/ethereum-2.0/eth-2.0-phases/). We focus currently on what is running at the moment i.e., ETH1.x



### Smart Contracts based on Ethereum

Etheruem in [one picture](https://github.com/kernoelpanic/BlockchainIllustrations/tree/master/Ethereum) (slightly outdated but still quite close):

<img src="https://raw.githubusercontent.com/kernoelpanic/BlockchainIllustrations/master/Ethereum/EthBlockchain5_bg.png" width=1024 height=768>

### Terminology and important references 

#### Ethereum(s)
There are several Ethereum(s) (due to community disagreement about the DAO Hack), but those are (still) technically more or less the same, dispite their blockchains differ since the DAO Hack : 
* [Ethereum](https://www.ethereum.org/)
    - we will refer to this Ethereum for the rest of the lecture
    - [wiki](https://eth.wiki/en/home)
    - [ethhub](https://docs.ethhub.io/)
    - [yellow paper](https://ethereum.github.io/yellowpaper/paper.pdf)
    - [yellow paper github](https://github.com/ethereum/yellowpaper) (really the current version)
    - [white paper](https://eth.wiki/en/white-Paper) (more decent introduction)
* [Ethereum Classic](https://ethereumclassic.github.io/)

#### Versions / Roadmap
The major versions of the Ethereum specification have dedicated names like: *Frontier*, *Homestead*, *Metropolis*, *Byzantium*, *Constantinople*, *Istanbul*,  ... 
* [Roadmap](https://eth.wiki/en/roadmap)

#### Clients
There are also several Ethereum clients e.g.,:
* [geth](https://github.com/ethereum/go-ethereum/wiki/geth)
    - written in go
    - we will use this client in the project/exercise
* [parity](https://github.com/paritytech/parity-ethereum)
    - written in rust
* [cpp-ethereum](https://github.com/ethereum/aleth/wiki)
    - written in cpp
* [pyethapp](https://github.com/ethereum/pyethapp)
    - written in python
    - mostly for dev. purposes

#### Testnets
There are several Ethereum networks / blockchains.

* Mainnet
    - web blockchain explorers:
        - [Etherscan](https://etherscan.io/)
        - [Etherchain](https://www.etherchain.org/)
* Public Testnets e.g.,:
    - *Ropsten* 
        - PoW, should be compatiple with all clients that also work on main chain
        - [Etherscan Ropsten explorer](https://ropsten.etherscan.io/)
    - *Morden*
        - Old public PoW testnet, now Ethereum classic testnet
    - *Rinkeby*
        - Public PoA geth compatible testnet
    - *Kovan*
        - Public PoA parity compatible testnet 
    
There are also private testnets. For the project/exercise we will use a
private geth PoA network to which you will connect your nodes. 

For the tutorial we will use a small development testnet provided by `ganache`

## Tutorial on Smart Contracts in Ethereuem 

### Basic interaction with `geth`,`parity` or `ganache` using web3py

The most common client lib to interface with ethereum clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node version
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - https://web3py.readthedocs.io/en/stable/

For the tutorial we will only interact with `ganache` which uses his own local development testnet blockchain. But all commands issues via `web3py` are the same when connected to a "real" `geth` or `parity` node.
* [ganache-cli](https://github.com/trufflesuite/ganache-cli/tree/master)

When running `geth` yout can also attach to it and use the JavaScript console (Read, Evaluate & Print Loop = REPL). 
Some stuff works in the geth console that does not work in `web3*`.
Once `geth` is running on localhost start the binary again to attach to the running instance:
```
geth attach "http://localhost:8545"
```

#### Connect
Import the web3 libarary and connect to a running `geth` or `ganache` node via `HTTPProvider`

For the tutorial configuration the docker container of ganache is located at `172.18.0.2`

In [31]:
import web3

In [32]:
w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.2:8545"))
# check if connection was successful
assert w3.isConnected()

If connected to `geth`,`parity` or as in our case to `ganache` you can execute serveral commands with the client to interact with the blockchain the client is sychronizing to. 



First it is always good the check to which blockchain the client is connected. In our case `ganache` starts a development chain with a random ID. The network ID provides information on which network the connected node resides.
The Ethereuem main network has id `1`.
Public test networks or private custom test networks should have different network IDs.
For a list of network IDs see this list:
* https://ethereum.stackexchange.com/questions/17051/how-to-select-a-network-id-or-is-there-a-list-of-network-ids

**Note:** Ethereuem and Ethereuem Classice have the same network ID! To avoid replay attacks of transactions the `CHAIN_ID` was introduced and is used in transactions (encoded into the signature creation, see below). 
* https://github.com/ethereum/EIPs/blob/master/EIPS/eip-155.md

In [33]:
# display network ID of client you are connected to
w3.net.version

'1575555942207'

Next it is interesting if the node is connected to other peers e.g., other `geth` clients.

For our tutorial `ganache` is not connected to any peer since we have our own local blockchain. Our w3 connection is not a peer! The w3 connection is just our interface to our client, therefore only local.

For the exercise your `geth`/`parity` node will connect to our server in the private PoA testnet. Then you should see at least one peer i.e., our server. For a connection to be successful there are two requirements:
* Both peers need to have the same **network ID** 
* Both peers need to have the same **genesis.json** / genesis block


In [34]:
# number of connected peers to our peer
w3.net.peerCount

0

For the exercise it will also be important to chech if your node is synchronized with the head of the blockchain i.e., our PoA node. 
If the node is started for the first time he will after some time start syncing. 
If syncing is *False* then you should see a `blockNumber` > 0

In [35]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3.eth.syncing

False

In [36]:
# return current blockchain head of node
# Since ganache "simulates" blokchain,
# blocks are mined on demand.
# Therefore, no block has been mined yet
w3.eth.blockNumber

0

In [42]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'number': 0,
 'hash': HexBytes('0xe203965c9eace3138398f707ea080ddb2dc636e5cda151e8a5b1f2d33370e96b'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

As you see, the genesis block is special as it is the only block with no parent hash:

In [44]:
genesis_block['number']

0

In [43]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

In [38]:
# instruct our test blockchain to mine a block
w3.provider.make_request('evm_mine',params='')

{'id': 6, 'jsonrpc': '2.0', 'result': '0x0'}

In [39]:
# for older web3py versions the command would be:
# w3.providers[0].make_request('evm_mine',params='')

In [40]:
w3.eth.blockNumber

1

In [45]:
first_block = w3.eth.getBlock(1)
first_block

AttributeDict({'number': 1,
 'hash': HexBytes('0xa9e4072d4b8bb2905fa91128596616d93bbbeba19972e4228cbb6a27032c7a94'),
 'parentHash': HexBytes('0xe203965c9eace3138398f707ea080ddb2dc636e5cda151e8a5b1f2d33370e96b'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [46]:
first_block['number']

1

In [47]:
first_block['parentHash']

HexBytes('0xe203965c9eace3138398f707ea080ddb2dc636e5cda151e8a5b1f2d33370e96b')

In [48]:
# our node is configured to mine blocks 
w3.eth.mining

True

In [49]:
# but since it is a fake chain we have no hashrate
w3.eth.hashrate

0

### Accounts and balances

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

`ganache` generates a list of accounts on startup. 
For the exercise you will recieve your personal account keypair which you have to copy in your `geth`/`parity` folder to import it. 

In [50]:
w3.eth.accounts

['0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 '0x22d491Bde2303f2f43325b2108D26f1eAbA1e32b',
 '0xE11BA2b4D45Eaed5996Cd0823791E0C93114882d',
 '0xd03ea8624C8C5987235048901fB614fDcA89b117',
 '0x95cED938F7991cd0dFcb48F0a06a40FA1aF46EBC',
 '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 '0x28a8746e75304c0780E011BEd21C72cD78cd535E',
 '0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e']

In [65]:
print("address length with 0x at the beginning = ",len(w3.eth.accounts[0]))
w3.eth.accounts[0]

address length with 0x at the beginning =  42


'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

**Note:** Ethereum uses Keccak-256 but often refers to it as SHA3, but the output of SHA3 as specified by NIST is different!

In [51]:
from sha3 import sha3_256
sha3_256(b'').hexdigest()

'a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a'

In [52]:
from sha3 import keccak_256
keccak_256(b'').hexdigest()

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [59]:
w3.sha3(text="").hex()[2:]

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [60]:
assert keccak_256(b'').hexdigest() == w3.sha3(text="").hex()[2:]

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase`

In [66]:
assert w3.eth.accounts[0] == w3.eth.coinbase
w3.eth.coinbase

'0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'

There is also a helper method to check if a string is an ethereum address.

In [30]:
w3.isAddress(w3.eth.accounts[0])

True

In [67]:
test_addr = '0xd3cda913deb6f67967b99d67acdfa1712c293601'
print(test_addr,"\n",len(test_addr))
print(test_addr[:-1],"\n",len(test_addr[:-1]))
w3.isAddress(test_addr[:-1])

0xd3cda913deb6f67967b99d67acdfa1712c293601 
 42
0xd3cda913deb6f67967b99d67acdfa1712c29360 
 41


False

In [68]:
w3.isAddress(test_addr)

True

**Note:** The capital letters of an address encode an optional checksum
https://github.com/ethereum/EIPs/blob/master/EIPS/eip-55.md

In [69]:
w3.isChecksumAddress(test_addr)

False

In [71]:
test_addr = w3.toChecksumAddress(test_addr)
print(test_addr)
w3.isChecksumAddress(test_addr)

0xd3CdA913deB6f67967B99D67aCDFa1712C293601


True

### Balance and gas
To query the balance of an account or contract the function `eth.getBalance()` can be used with the respective address.

**Note:** Per default all balances are in *Wei* i.e., the smallest unit of value in Ethereum. 
$$
    1 \textrm{ ether} = 10^{18} \textrm{ wei} 
$$

In [72]:
balance = w3.eth.getBalance(w3.eth.accounts[0])
balance

100000000000000000000

In [73]:
balance * 10**-18

100.00000000000001

In [74]:
# Directly output the balance in ether
w3.fromWei(balance,'ether')

Decimal('100')

In [75]:
def getBalance(address):
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [76]:
getBalance(w3.eth.accounts[0])

Decimal('100')

In [77]:
getBalance(w3.eth.accounts[1])

Decimal('100')

We now send a transaction that just transfers value i.e., ether from one account to another. Thereby, `ganache` automatically mines a block once he sees that a transactions requires confirmation. 

**Note:** When connected to `geth` or `parity` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `ganache` this is not necessary since it is a development environment.

In [80]:
# required in geth but not in ganache
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"",0)

True

In [81]:
# old general way now client may be different.
#w3.personal.unlockAccount(w3.eth.accounts[0],"",0)

In [82]:
w3.eth.blockNumber

1

In [83]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c')

In [84]:
blkNumber = w3.eth.blockNumber
blkNumber

2

In [87]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
getBalance(w3.eth.accounts[0])

Decimal('98.99958')

In [88]:
getBalance(w3.eth.accounts[1])

Decimal('101')

In [89]:
# get first (and only) transaction in the previously mined block
w3.eth.getTransactionByBlock(blkNumber,0)

AttributeDict({'hash': HexBytes('0xbfda5a53139f0066893d3d3947df833807b90b0ab567494d1cc21b0c42b7b04c'),
 'nonce': 0,
 'blockHash': HexBytes('0x28d5623c6cf6a3fdeb1afdc09f57dbc57a6aaa929e835e8a8c2f56652f6d05ee'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 'value': 1000000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 28,
 'r': HexBytes('0x7c95c2cac783bb99be1314d3f3fbf41d1b3befb550d9c469e2378ce9a537d942'),
 's': HexBytes('0x6cdc83acacfd8d71721df2e24907db8bb04fc81306cc4922b818b294e8732eaf')})

In [48]:
# get block and see the transaction id and gas used below
blk = w3.eth.getBlock(blkNumber)
blk

AttributeDict({'number': 2,
 'hash': HexBytes('0x7abce3c20ab7354fac837c8443a8e27c44807584e8c4db15161211cc205887c9'),
 'parentHash': HexBytes('0xb13ca902ab0c4b7344e0339ba511f21221173a8797fdba5e173a55968776b4c2'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [49]:
# estimate the gas price of an transaction without executing it
w3.eth.estimateGas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

In [50]:
# conversion between gas and ether specified per transaction
# i.e., how much wei a transaction is willing to pay for 
w3.eth.gasPrice

20000000000

In [51]:
# gasUsed(by the transaction) * gasPrice(specified in tx)
# gives the wei it costs to send it 
# times 10^18 gives the ether
(21000 * 20000000000) * 10**-18

0.00042

In [52]:
# get the transaction ID of the first (and only) transaction in the block
blk["transactions"][0].hex()

'0x20450f54446ff6efcba218d00ebad3ab8f35dfc7364604cc9474daaf7053efd0'

In [53]:
# Query the transaction by ID
w3.eth.getTransaction(blk["transactions"][0].hex())

AttributeDict({'hash': HexBytes('0x20450f54446ff6efcba218d00ebad3ab8f35dfc7364604cc9474daaf7053efd0'),
 'nonce': 0,
 'blockHash': HexBytes('0x7abce3c20ab7354fac837c8443a8e27c44807584e8c4db15161211cc205887c9'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
 'to': '0x636398E0834B5f49bd550768A7E8384CE186a704',
 'value': 1000000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 27,
 'r': HexBytes('0xc61973eccdecfd9c271b19456af96b5bb2a49770393c675125f302c5c7205769'),
 's': HexBytes('0x4f109a2d430886e7c4f0821e6629a417c70a617d52e564cc9ff411e636e11969')})

## Contract deployment and interaction (the hard way)

Now its time to deploy our first contract.
For the first time we do it the hard way.

In [54]:
# execute console command and print Greeter.sol
!cat Greeter/Greeter.sol

pragma solidity ^0.4.25;

contract Greeter {
    string public greeting;
    uint256 public greetbit;

    //Old constructor syntax, function name same as contract name:
    //function Greeter() public {
    //new constructor syntax:
    constructor() public {
        greeting = 'Hello';
    }

    function setGreeting(string _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string) {
        return greeting;
    }

    function setGreetbit(uint256 bit) public {
        greetbit = bit;
    }

    function greetbit() view public returns (uint256) {
        return greetbit; 
    }

    function() public payable{
        greetbit = greetbit ^ 1;
    }

}


In [55]:
cd Greeter 

/smartcode/Greeter


In [56]:
!pwd

/smartcode/Greeter


In [57]:
!solc --help

solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also possible to
remap paths using the context:prefix=path syntax.
Example:
    solc --bin -o /tmp/solcoutput dapp-bin=/usr/local/lib/dapp-bin contract.sol

Allowed options:
  --help               Show help message and exit.
  --version            Show version and exit.
  --license            Show licensing information and exit.
  --evm-version version
                       Select desired EVM version. Either homestead, 
                       tangerine

In [58]:
!solc --bin --overwrite -o ./ Greeter.sol

In [59]:
!cat Greeter.bin

608060405234801561001057600080fd5b506040805190810160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b61049a806101166000396000f30060806040526004361061006d576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff1680631c55ad301461007a5780634f11dbea146100a5578063a4136862146100d2578063cfae32171461013b578063ef690cc0146101cb575b6001805418600181905550005b34801561008657600080fd5b5061008f61025b565b6040518082815260200191505060405180910390f35b3480156100b157600080fd5b506100d060048036038101908080359060200190929190505050610265565b005b3480156100de57600080fd

In [60]:
# assign cat output to python variable
out = !cat Greeter.bin
cbin = out[0]

In [61]:
!solc --abi --overwrite -o ./ Greeter.sol

In [62]:
!cat Greeter.abi

[{"constant":true,"inputs":[],"name":"greetbit","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"bit","type":"uint256"}],"name":"setGreetbit","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"greeting","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":true,"stateMutability":"payable","type":"fallback"}]

In [63]:
out = !cat Greeter.abi
cabi = out[0]

One way to send a deploy transaction is using the w3py libary helper function cunstructor.

In [64]:
# send transaction to deploy contract with helper function
contract=w3.eth.contract(abi=cabi,
                         bytecode=cbin)
tx_hash = contract.constructor().transact({"from":w3.eth.accounts[0],})
tx_hash

HexBytes('0x2f739639a56df99ece275a6dce7df6494bbc9f7b1a1cbeb67a7d8d6761785ebd')

Another way which can also estimate the gas costs of the deploy transaction is sending is 
directly via `sendTransaction` and add the previously compiled contract as `data`

In [65]:
# estimate gas of deployment transaction
w3.eth.estimateGas({"from":w3.eth.accounts[0],"data":cbin,"value":0})

400812

In [66]:
# send transaction and get transaction hash
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"data":cbin,"value":0})
tx_hash

HexBytes('0x1c834209ae88f1419edff0cd748ed1c14136e868d297740158ccdc0f83b4bd1d')

In [67]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x1c834209ae88f1419edff0cd748ed1c14136e868d297740158ccdc0f83b4bd1d'),
 'nonce': 2,
 'blockHash': HexBytes('0xf8127e865cf9fdf896c3ef62bcfbf5ecf31fdfe758baee461ab424e928489b28'),
 'blockNumber': 4,
 'transactionIndex': 0,
 'from': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
 'to': None,
 'value': 0,
 'gas': 500812,
 'gasPrice': 20000000000,
 'input': '0x608060405234801561001057600080fd5b506040805190810160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b61049a806101166000396000f30060806040526004361061006d576000357c010000000000000000000

Normally one has to wait till the transaction goes through, with ganache this is almost instant 

In [68]:
# wait till tx goes through 
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

# other way without wait only successful if mined:
# w3.eth.getTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x1c834209ae88f1419edff0cd748ed1c14136e868d297740158ccdc0f83b4bd1d'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xf8127e865cf9fdf896c3ef62bcfbf5ecf31fdfe758baee461ab424e928489b28'),
 'blockNumber': 4,
 'from': '0xf79bf39ced6798f751ee8949bbf70979dff9aec5',
 'to': None,
 'gasUsed': 400812,
 'cumulativeGasUsed': 400812,
 'contractAddress': '0xC3FFc3F7092CF9b38617E88aA82d6c567Bd035EF',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [69]:
# the important thing from the tx receipt is the address at which 
# the contract was deployed
caddress = tx_receipt["contractAddress"]
caddress

'0xC3FFc3F7092CF9b38617E88aA82d6c567Bd035EF'

In [70]:
# double check if the gasUsed was really our estimate
w3.eth.getBlock(tx_receipt["blockNumber"])

AttributeDict({'number': 4,
 'hash': HexBytes('0xf8127e865cf9fdf896c3ef62bcfbf5ecf31fdfe758baee461ab424e928489b28'),
 'parentHash': HexBytes('0xdf60598f9903a72aff09faa63b98059be57fedfdf3c02e9394ee16c8aaf67808'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

### Interact with deployed contract

To get an instance of a contract at its current state we require:
* The contract address `caddress`
* Some ABI information `cabi` either from compiling the contract like done previously or reconstructed through reverse engineering the bytecode stored in the blockchain.  

In [71]:
instance = w3.eth.contract(address=caddress,
                           abi=cabi)

In [72]:
instance.address

'0xC3FFc3F7092CF9b38617E88aA82d6c567Bd035EF'

In [73]:
instance.abi

[{'constant': True,
  'inputs': [],
  'name': 'greetbit',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': 'bit', 'type': 'uint256'}],
  'name': 'setGreetbit',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': '_greeting', 'type': 'string'}],
  'name': 'setGreeting',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greet',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greeting',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'co

In [74]:
# view functions dont change state and can therefore be executed 
# without performing a transaction on the blockchain
instance.functions.greet().call()

'Hello'

In [75]:
tx_hash = instance.functions.setGreeting("Hello World!").transact(
    {"from":w3.eth.accounts[0]})
tx_hash

HexBytes('0xf0eaf515f648ebc34d932ff6e095d93e1ddbf78e237915c9cbeb70d412604c2a')

In [76]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [77]:
# lets check if the output is now different i.e., the value has changed 
instance.functions.greet().call()

'Hello World!'

In [78]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0xf0eaf515f648ebc34d932ff6e095d93e1ddbf78e237915c9cbeb70d412604c2a'),
 'nonce': 3,
 'blockHash': HexBytes('0xc910fa861cbaa3b798b2e7a343d7ab08d3ff8b82c0f2a5b152edb435d8ec0410'),
 'blockNumber': 5,
 'transactionIndex': 0,
 'from': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
 'to': '0xC3FFc3F7092CF9b38617E88aA82d6c567Bd035EF',
 'value': 0,
 'gas': 133560,
 'gasPrice': 20000000000,
 'input': '0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000',
 'v': 27,
 'r': HexBytes('0x3a6f6ee7b4a7bbec9f9d68c5b470e27b543f13196eefa8c0b7980038a1cc9384'),
 's': HexBytes('0x1f32d3aa577be192a6629813d3a91f37938da77e9ab29c0d30ebb55714aad389')})

In [79]:
# lets decode the input data of the transaction
input_data = bytes.fromhex("48656c6c6f20576f726c6421").decode("utf-8")
print(input_data) # input data string

Hello World!


In [80]:
# there is also a helper function that can do this
instance.decode_function_input(tx_info.input)

(<Function setGreeting(string)>, {'_greeting': 'Hello World!'})

In [81]:
# There is also a way to directly access the storage of a contract
w3.eth.getStorageAt(caddress,0)

HexBytes('0x48656c6c6f20576f726c64210000000000000000000000000000000000000018')

### Function signature / method id
The four byte *function signature* or *method id* has to be specified at the beginning of the input data. It selects the function that is invoked with the respective arguments.

In [82]:
# 4 byte function signature or method_id
len("a4136862")/2.

4.0

In [83]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreeting(string)").hexdigest()
method_id = sha3_hash[:8]
method_id

'a4136862'

In [84]:
instance.functions.greetbit().call()

0

In [85]:
tx_hash = instance.functions.setGreetbit(1).transact(
    {"from":w3.eth.accounts[0]})
tx_hash

HexBytes('0x8d52457fb4d423afde3e6d5da58dfd414aa8c95720a1c7a02388a6430963cdcd')

In [86]:
tx_info = w3.eth.getTransaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x8d52457fb4d423afde3e6d5da58dfd414aa8c95720a1c7a02388a6430963cdcd'),
 'nonce': 4,
 'blockHash': HexBytes('0xd6167d326d3cb73d94b94dd5c6c577e4b5cdd05acc775b2f122f5821fe00db96'),
 'blockNumber': 6,
 'transactionIndex': 0,
 'from': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
 'to': '0xC3FFc3F7092CF9b38617E88aA82d6c567Bd035EF',
 'value': 0,
 'gas': 141691,
 'gasPrice': 20000000000,
 'input': '0x4f11dbea0000000000000000000000000000000000000000000000000000000000000001',
 'v': 27,
 'r': HexBytes('0xbdaaeae74f1721f9acc9bd9220c258722676ce7209448d83e42efc2347f37e14'),
 's': HexBytes('0x6546d5af2cb0ec5599a5ac5575492ea82ed70026e22b4e04406022f3150e9ff3')})

In [87]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreetbit(uint256)").hexdigest()
method_id = sha3_hash[:8]
method_id

'4f11dbea'

In [88]:
len("0000000000000000000000000000000000000000000000000000000000000001")/2 * 8

256.0

You can also manually send a transaction, this should work with `geth`

In [89]:
# This should be possible with geth but is not with web3py:
cdata = b"0x4f11dbea0000000000000000000000000000000000000000000000000000000000000000"
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":0,"gas":1_000_000})
tx_hash

HexBytes('0xad4bc4aa78a4b6309abbc43cf0414eec6126e34900fedaa7eb5e92498d8d38d2')

In [90]:
instance.functions.greetbit().call()

0

You can also use the `getData()` method in the `geth` REPL console to construct the `input` data hex string. 

In [91]:
# debug i.e. trace a transaction and its operations 
w3.providers[0].make_request("debug_traceTransaction", [tx_hash.hex(), {}])

{'id': 79,
 'jsonrpc': '2.0',
 'result': {'gas': 31425,
  'returnValue': '',
  'structLogs': [{'depth': 0,
    'error': '',
    'gas': 973968,
    'gasCost': 26032,
    'memory': [],
    'op': 'PUSH1',
    'pc': 0,
    'stack': [],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 973965,
    'gasCost': 3,
    'memory': [],
    'op': 'PUSH1',
    'pc': 2,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080'],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 973962,
    'gasCost': 3,
    'memory': [],
    'op': 'MSTORE',
    'pc': 4,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080',
     '0000000000000000000000000000000000000000000000000000000000000040'],
    'storage': {}},
   {'depth': 0,
    'error': '',
    'gas': 973950,
    'gasCost': 12,
    'memory': ['0000000000000000000000000000000000000000000000000000000000000000',
     '0000000000000000000000000000000000000000000000000000000000000000',
 

### Fallback function

The fallback function is the function that is invoked when the method ID does not match any function or then ether is sent directly to the contract. For a contract to be able to accept ether this function has to be defined as `payable`. 

In [92]:
instance.functions.greetbit().call()

0

In [93]:
cdata = b"0xffffffff"
tx_hash = w3.eth.sendTransaction({"from":w3.eth.accounts[0],"to":caddress,"value":0})
tx_hash

HexBytes('0xb3bf80832cacc5e85a900e48108f0808c08d3371996468a13e06981fa4a4cd13')

In [94]:
instance.functions.greetbit().call()

1

## Contract deployment and interaction (the easy way) 

Now the easy way using a small utility lib

In [95]:
pwd

'/smartcode/Greeter'

In [96]:
cat Greeter/Greeter.sol

cat: Greeter/Greeter.sol: No such file or directory


In [97]:
import myutil as util

In [98]:
instance = util.compile_and_deploy_contract("/smartcode/Greeter/Greeter.sol")

In [99]:
instance.address

'0xF4495882E89e1F353fc33a74224140Ec572Ad1c9'

In [100]:
instance.greet()

'Hello'

In [101]:
# With the patched concise contract Factory the function
# waits till a block is mined and directly returns the tx receipt
tx_receipt = instance.setGreeting("Nihao", transact={"from": w3.eth.accounts[0]})
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x906169996be13a6ce51b3029c033b7136f38058ec85d8817315194ea0db9b904'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x794096279fb3c559e500b0aa9e80b9f40444f90fad778967b3afa552ad15ecfc'),
 'blockNumber': 10,
 'from': '0xf79bf39ced6798f751ee8949bbf70979dff9aec5',
 'to': '0xf4495882e89e1f353fc33a74224140ec572ad1c9',
 'gasUsed': 33112,
 'cumulativeGasUsed': 33112,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [102]:
instance.greet()

'Nihao'

## Events

Events are a way to notify clients listening to new blocks about smart contract operations that have been performed.
* https://solidity.readthedocs.io/en/develop/contracts.html#events
* https://web3py.readthedocs.io/en/stable/filters.html?highlight=events
* https://web3py.readthedocs.io/en/stable/contracts.html#events

In [103]:
pwd

'/smartcode/Greeter'

In [104]:
cd smartcode

[Errno 2] No such file or directory: 'smartcode'
/smartcode/Greeter


In [105]:
!cat Eventer/Eventer.sol

cat: Eventer/Eventer.sol: No such file or directory


In [106]:
!solc --bin Eventer/Eventer.sol

""Eventer/Eventer.sol"" is not found


In [107]:
# create a contract instance 
einstance = util.compile_and_deploy_contract("/smartcode/Eventer/Eventer.sol")

In [108]:
einstance.address

'0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2'

Now we create a listener to listen for the latest *create* events and return all of them.

In [109]:
event_filter = einstance.events.create.createFilter(fromBlock='latest')

In [110]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'v': 1}),
  'event': 'create',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x971a2e47b46428d2da95b56bb9a49f94bff9f9a9a236e114d90e00b7d1a2e6cb'),
  'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
  'blockHash': HexBytes('0x92f98ca241121ce8de5878cd9552881669ce6277e673b3d7c593517a0522a134'),
  'blockNumber': 11})]

Now we fire a new event by creating a new transaction and keep the `tx_receipt` which we cen process later on. 

In [111]:
tx_receipt = einstance.func1(3,transact={"from":w3.eth.coinbase})
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xa3a8dc3c6ffd63506b8b0d0755954b0345886498a29c042708cf3305329cda61'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xf79c88516a9b20f8c63efcf6f5d54558eec9fa0537e9f69dbc2f20faaf455e63'),
 'blockNumber': 12,
 'from': '0xf79bf39ced6798f751ee8949bbf70979dff9aec5',
 'to': '0xbbfc344087fd2b285b5ca8798f66d8d7e43925d2',
 'gasUsed': 23034,
 'cumulativeGasUsed': 23034,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xa3a8dc3c6ffd63506b8b0d0755954b0345886498a29c042708cf3305329cda61'),
   'blockHash': HexBytes('0xf79c88516a9b20f8c63efcf6f5d54558eec9fa0537e9f69dbc2f20faaf455e63'),
   'blockNumber': 12,
   'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
   'data': '0x000000000000000000000000f79bf39ced6798f751ee8949bbf70979dff9aec50000000000000000000000000000000000000000000000000000000000000003',
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3

The `tx_receipt` contains the respective event.

In [112]:
einstance.events.funcall().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xa3a8dc3c6ffd63506b8b0d0755954b0345886498a29c042708cf3305329cda61'),
  'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
  'blockHash': HexBytes('0xf79c88516a9b20f8c63efcf6f5d54558eec9fa0537e9f69dbc2f20faaf455e63'),
  'blockNumber': 12}),)

The `tx_receipt` does **not** contain the respective event.

In [113]:
einstance.events.funcall2().processReceipt(tx_receipt)

()

Now lets fire the `funcall` event again collect all events until now

In [114]:
tx_receipt = einstance.func1(3,transact={"from":w3.eth.coinbase})
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xe540ce889d70bb063b03ee54cb0cd08e83b8f7fbe787978fca784f3b270a48de'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x9b76f8330413d933d5471890f8b771136c4ecf03f2b3386934f92fa127857b9e'),
 'blockNumber': 13,
 'from': '0xf79bf39ced6798f751ee8949bbf70979dff9aec5',
 'to': '0xbbfc344087fd2b285b5ca8798f66d8d7e43925d2',
 'gasUsed': 23034,
 'cumulativeGasUsed': 23034,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xe540ce889d70bb063b03ee54cb0cd08e83b8f7fbe787978fca784f3b270a48de'),
   'blockHash': HexBytes('0x9b76f8330413d933d5471890f8b771136c4ecf03f2b3386934f92fa127857b9e'),
   'blockNumber': 13,
   'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
   'data': '0x000000000000000000000000f79bf39ced6798f751ee8949bbf70979dff9aec50000000000000000000000000000000000000000000000000000000000000003',
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3

In [115]:
event_filter = einstance.events.funcall.createFilter(fromBlock=0)

In [116]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'caller': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xa3a8dc3c6ffd63506b8b0d0755954b0345886498a29c042708cf3305329cda61'),
  'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
  'blockHash': HexBytes('0xf79c88516a9b20f8c63efcf6f5d54558eec9fa0537e9f69dbc2f20faaf455e63'),
  'blockNumber': 12}),
 AttributeDict({'args': AttributeDict({'caller': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xe540ce889d70bb063b03ee54cb0cd08e83b8f7fbe787978fca784f3b270a48de'),
  'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
  'blockHash': HexBytes('0x9b76f8330413d933d5471890f8b771136c4ecf03f2b3386934f92fa127857b9e'),
  'blockNumber': 13})]

In [117]:
tx_hash = w3.eth.sendTransaction({"from":w3.eth.coinbase,
                                  "to":einstance.address,
                                  "value":1,
                                  "gas":1_000_000})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xb0d1d3b9997572d85773fdab5b44fbcfb09ab053b8a994c00be869fa3fcc23b7'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x3ef55a70464ec58ac7634d4ca080cebc4c36bee0a9536d4982f0082ec9c5db55'),
 'blockNumber': 14,
 'from': '0xf79bf39ced6798f751ee8949bbf70979dff9aec5',
 'to': '0xbbfc344087fd2b285b5ca8798f66d8d7e43925d2',
 'gasUsed': 22118,
 'cumulativeGasUsed': 22118,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xb0d1d3b9997572d85773fdab5b44fbcfb09ab053b8a994c00be869fa3fcc23b7'),
   'blockHash': HexBytes('0x3ef55a70464ec58ac7634d4ca080cebc4c36bee0a9536d4982f0082ec9c5db55'),
   'blockNumber': 14,
   'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
   'data': '0x000000000000000000000000f79bf39ced6798f751ee8949bbf70979dff9aec5',
   'topics': [HexBytes('0x558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d')],
   'type': 'mined'})],
 'status': 1,
 'logsB

In [118]:
# get the fallback function event
einstance.events.fallcall().processReceipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0xf79bf39CEd6798f751eE8949BbF70979dFf9AEC5'}),
  'event': 'fallcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xb0d1d3b9997572d85773fdab5b44fbcfb09ab053b8a994c00be869fa3fcc23b7'),
  'address': '0xBBFC344087fD2b285B5cA8798f66d8d7E43925D2',
  'blockHash': HexBytes('0x3ef55a70464ec58ac7634d4ca080cebc4c36bee0a9536d4982f0082ec9c5db55'),
  'blockNumber': 14}),)

## Visibility

Generally all data stored in the blockchain is visible!
Function invocations can be restricted but be careful!

* https://solidity.readthedocs.io/en/v0.4.25/contracts.html#visibility-and-getters

In [125]:
cd /

/


In [126]:
cd /smartcode

/smartcode


In [127]:
!cat Visibility/Visibility.sol

pragma solidity ^0.4.25;

contract Visibility {
    uint256 constant public cpub_int = 0x10; //set at compile time
    uint256 public pub_int;   //getter is created automatically 
    uint256 internal int_int; //no getter but available in derived contracts 
    uint256 private priv_int;

    event echange(uint256 indexed v);
    event ichange(uint256 indexed v);
    event pchange(uint256 indexed v);

    constructor() public payable {
        pub_int = 0x20;
        int_int = 0x30;
        priv_int = 0x40;
    }
    
    // Function _should_ only be called from external accounts/contracts
    // Although can also be called form this contract with this.
    function ext_change() external returns (uint256) {
        priv_int += 1;
        emit echange(priv_int);
        return priv_int;
    }
    
    // Internal functions cannot be called directly from external accounts/contracts
    // Only indirectly 
    function int_change() internal returns (uint256) {
        int_int += 1;
       

In [128]:
!solc --bin Visibility/Visibility.sol

Visibility/Visibility.sol:48:9: Warning: Function declared as view, but this expression (potentially) modifies the state and thus requires non-payable (the default) or payable.
        priv_int = v;
        ^------^

======= Visibility/Visibility.sol:Visibility =======
Binary: 
60806040526020600081905550603060018190555060406002819055506103878061002b6000396000f300608060405260043610610078576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff1680630f66c3d21461007d5780634c370d25146100be578063529f246f146100e9578063ebd3b3dd14610114578063f455a8151461013f578063fcce02071461016a575b600080fd5b34801561008957600080fd5b506100a8600480360381019080803590602001909291905050506101ab565b6040518082815260200191505060405180910390f35b3480156100ca57600080fd5b506100d36101be565b6040518082815260200191505060405180910390f35b3480156100f557600080fd5b506100fe610208565b6040518082815260200191505060405180910390f35b34801561012057600080fd5b506101296102fa565b604051808281526020019150506040518091039

In [129]:
vinstance = util.compile_and_deploy_contract("./Visibility/Visibility.sol")

In [130]:
hex(vinstance.pub_int())

'0x20'

In [131]:
hex(vinstance.pub_change())

'0x21'

In [132]:
hex(vinstance.view_priv_int(1))

'0x1'

In [133]:
tx_receipt = vinstance.pub_change(transact={"from":w3.eth.coinbase,})
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xa9fee13e0ecd6778a029a90d80be8278dfc0eb7d4063cc5bdc731d5e5cba954e'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x7f90ce8e42e2b0258d9a6f3d943d882cb0d633dc179c83b29a5c73ad9532c948'),
 'blockNumber': 16,
 'from': '0xf79bf39ced6798f751ee8949bbf70979dff9aec5',
 'to': '0xceac1fa5968abb352f56cb9442ec32bc15c16458',
 'gasUsed': 43761,
 'cumulativeGasUsed': 43761,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xa9fee13e0ecd6778a029a90d80be8278dfc0eb7d4063cc5bdc731d5e5cba954e'),
   'blockHash': HexBytes('0x7f90ce8e42e2b0258d9a6f3d943d882cb0d633dc179c83b29a5c73ad9532c948'),
   'blockNumber': 16,
   'address': '0xcEaC1Fa5968AbB352f56CB9442EC32bC15c16458',
   'data': '0x',
   'topics': [HexBytes('0x437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538e1def345'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000041')],
   'type': 'mined'}),
  